In [1]:
# header files
import numpy as np
import cv2
from skimage.feature import local_binary_pattern
from skimage.color import rgb2gray
from skimage import io

print("Header files loaded")

Header files loaded


In [92]:
# define image path
image_path = "../../sample_patches_ovarian_cancer_data/TCGA-25-2401_14000_38000.png"
epi_stroma_mask_path = "../../sample_patches_ovarian_cancer_data/epithelium_stroma_masks/TCGA-25-2401_14000_38000_epi_stroma_mask.png"
nuclei_mask_path = "../../sample_patches_ovarian_cancer_data/nuclei_masks/TCGA-25-2401_14000_38000.png"

In [93]:
# hyperparameters
lbp_radius = 3
lbp_numpoints = 9

In [94]:
# read image and convert it to grayscale
image = io.imread(image_path)
image_gray = rgb2gray(image)


# create image empty mask for removing collagen from empty area
image_empty = np.ones((image.shape[0], image.shape[1]))*255
for index1 in range(0, image.shape[0]):
    for index2 in range(0, image.shape[1]):
        if image[index1, index2, 0]>200 and image[index1, index2, 1]>200 and image[index1, index2, 2]>200:
            image_empty[index1, index2] = 0
            
ret, image_empty_orig = cv2.threshold(image_empty, 200, 255, cv2.THRESH_BINARY)
output_image_empty = cv2.connectedComponentsWithStats(np.uint8(image_empty_orig), 8, cv2.CV_32S)
(num_labels, labels, stats, centroids) = output_image_empty            
image_empty_mask = np.zeros(image_empty.shape, dtype="uint8")
for index in range(1, num_labels):
    w = stats[index, cv2.CC_STAT_WIDTH]
    h = stats[index, cv2.CC_STAT_HEIGHT]
    if w>50 or h>50:
        componentMask = (labels == index).astype("uint8")*255
        image_empty_mask = cv2.bitwise_or(image_empty_mask, componentMask)
        
ret, image_empty_inv = cv2.threshold(image_empty_mask, 200, 255, cv2.THRESH_BINARY_INV)
output_image_empty_inv = cv2.connectedComponentsWithStats(np.uint8(image_empty_inv), 8, cv2.CV_32S)
(num_labels, labels, stats, centroids) = output_image_empty_inv            
image_empty_mask_inv = np.zeros(image_empty_inv.shape, dtype="uint8")
for index in range(1, num_labels):
    w = stats[index, cv2.CC_STAT_WIDTH]
    h = stats[index, cv2.CC_STAT_HEIGHT]
    if w>50 or h>50:
        componentMask = (labels == index).astype("uint8")*255
        image_empty_mask_inv = cv2.bitwise_or(image_empty_mask_inv, componentMask)
ret, image_empty_mask = cv2.threshold(image_empty_mask_inv, 200, 255, cv2.THRESH_BINARY_INV)


# print shapes of the images
print(image.shape)
print(image_gray.shape)
print(image_empty_mask.shape)

(2000, 2000, 3)
(2000, 2000)
(2000, 2000)


In [95]:
# read image and then convert to grayscale
image = io.imread(image_path)
image_gray = rgb2gray(image)


# create image red mask and remove collagen from red area
image_red = np.ones((image.shape[0], image.shape[1]))*255
for index1 in range(0, image.shape[0]):
    for index2 in range(0, image.shape[1]):
        if image[index1, index2, 0]>200 and image[index1, index2, 1]<50 and image[index1, index2, 2]<50:
            image_red[index1, index2] = 0
            
ret, image_red_orig = cv2.threshold(image_red, 200, 255, cv2.THRESH_BINARY)
output_image_red = cv2.connectedComponentsWithStats(np.uint8(image_red_orig), 8, cv2.CV_32S)
(num_labels, labels, stats, centroids) = output_image_red
image_red_mask = np.zeros(image_red.shape, dtype="uint8")
for index in range(1, num_labels):
    w = stats[index, cv2.CC_STAT_WIDTH]
    h = stats[index, cv2.CC_STAT_HEIGHT]
    if w>50 or h>50:
        componentMask = (labels == index).astype("uint8")*255
        image_red_mask = cv2.bitwise_or(image_red_mask, componentMask)
        
ret, image_red_inv = cv2.threshold(image_red_mask, 200, 255, cv2.THRESH_BINARY_INV)
output_image_red_inv = cv2.connectedComponentsWithStats(np.uint8(image_red_inv), 8, cv2.CV_32S)
(num_labels, labels, stats, centroids) = output_image_red_inv            
image_red_mask_inv = np.zeros(image_red_inv.shape, dtype="uint8")
for index in range(1, num_labels):
    w = stats[index, cv2.CC_STAT_WIDTH]
    h = stats[index, cv2.CC_STAT_HEIGHT]
    if w>50 or h>50:
        componentMask = (labels == index).astype("uint8")*255
        image_red_mask_inv = cv2.bitwise_or(image_red_mask_inv, componentMask)
ret, image_red_mask = cv2.threshold(image_red_mask_inv, 200, 255, cv2.THRESH_BINARY_INV)


# print shapes of the images
print(image.shape)
print(image_gray.shape)
print(image_red_mask.shape)

(2000, 2000, 3)
(2000, 2000)
(2000, 2000)


In [96]:
# get the corresponding epi region and remove collagen fiber from there
epi_stroma_mask = io.imread(epi_stroma_mask_path)
ret, epi_stroma_mask = cv2.threshold(epi_stroma_mask, 200, 255, cv2.THRESH_BINARY_INV)
print(epi_stroma_mask.shape)

(2000, 2000)


In [97]:
# get the corresponding nuclei region and remove collagen fiber from there
nuclei_mask = io.imread(nuclei_mask_path)
ret, nuclei_mask = cv2.threshold(nuclei_mask, 200, 255, cv2.THRESH_BINARY_INV)
print(nuclei_mask.shape)

(2000, 2000)


In [98]:
# apply local binary pattern texture descriptor
lbp_image_gray = local_binary_pattern(image_gray, lbp_numpoints, lbp_radius, "ror")


# post processing: thresholding and then apply connected component analysis
ret, lbp_image_gray = cv2.threshold(lbp_image_gray, 200, 255, cv2.THRESH_BINARY)
output = cv2.connectedComponentsWithStats(np.uint8(lbp_image_gray), 8, cv2.CV_32S)
(num_labels, labels, stats, centroids) = output


# form collagen mask
collagen_mask = np.zeros(lbp_image_gray.shape, dtype="uint8")
for index in range(1, num_labels):
    w = stats[index, cv2.CC_STAT_WIDTH]
    h = stats[index, cv2.CC_STAT_HEIGHT]
    if w>7 or h>7:
        componentMask = (labels == index).astype("uint8")*255
        collagen_mask = cv2.bitwise_or(collagen_mask, componentMask)
        

# print collagen mask shape
print(collagen_mask.shape)

(2000, 2000)


In [99]:
# remove collagen fibers from empty region, red region, epi region and stroma nuclei region
collagen_mask = cv2.bitwise_and(collagen_mask, image_empty_mask)
collagen_mask = cv2.bitwise_and(collagen_mask, image_red_mask)
collagen_mask = cv2.bitwise_and(collagen_mask, epi_stroma_mask)
collagen_mask = cv2.bitwise_and(collagen_mask, nuclei_mask)


# get final collagen mask
ret, collagen_mask = cv2.threshold(collagen_mask, 200, 255, cv2.THRESH_BINARY)
final_output = cv2.connectedComponentsWithStats(np.uint8(collagen_mask), 8, cv2.CV_32S)
(num_labels, labels, stats, centroids) = final_output


# form collagen mask
final_collagen_mask = np.zeros(collagen_mask.shape, dtype="uint8")
for index in range(1, num_labels):
    w = stats[index, cv2.CC_STAT_WIDTH]
    h = stats[index, cv2.CC_STAT_HEIGHT]
    if w>7 or h>7:
        componentMask = (labels == index).astype("uint8")*255
        final_collagen_mask = cv2.bitwise_or(final_collagen_mask, componentMask)
        
        
# print final collagen mask shape
print(final_collagen_mask.shape)

(2000, 2000)


In [ ]:
cv2.imwrite("../results/collagen_case_3.png", final_collagen_mask)

In [100]:
# overlay image
collagen_mask = cv2.cvtColor(final_collagen_mask, cv2.COLOR_GRAY2BGR)
image_updated = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
collagen_image_overlay = cv2.addWeighted(collagen_mask, 0.8, image_updated, 1, 0.0)


# print collagen_image_overlay shape
print(collagen_image_overlay.shape)

(2000, 2000, 3)


In [101]:
cv2.imwrite("../../sample_patches_ovarian_cancer_data/collagen_masks/TCGA-25-2401_14000_38000.png", collagen_image_overlay)

True